In [2]:
#!/usr/bin/env python

'''
Mean field with k-points sampling when Brillouin Zone symmetry is considered
'''

import numpy
from pyscf.pbc import gto, scf, dft

cell = gto.M(
    a = numpy.asarray([[0.0, 2.6935121974, 2.6935121974], 
                       [2.6935121974, 0.0, 2.6935121974], 
                       [2.6935121974, 2.6935121974, 0.0]]),
    atom = '''Si  0.0000000000 0.0000000000 0.0000000000
              Si  1.3467560987 1.3467560987 1.3467560987''', 
    basis = 'gth-szv',
    pseudo = 'gth-pade',
    verbose = 5,
    space_group_symmetry = True,
    #symmorphic: if True, only the symmorphic subgroup is considered
    symmorphic = True,
)

nk = [2,2,2]
#The Brillouin Zone symmetry info is contained in the kpts object
kpts = cell.make_kpts(nk, 
                      space_group_symmetry=True, 
                      time_reversal_symmetry=True)
print(kpts)

# kmf = scf.KRHF(cell, kpts)
# kmf.kernel()

# kmf = dft.KRKS(cell, kpts)
# kmf.xc = 'camb3lyp'
# kmf.kernel()

# By default, the mean-field calculation will use symmetry-adapted
# crystalline orbitals whenever possible. This can be turned off manually
# when instantiating the mean-field object.
kmf = scf.KRHF(cell, kpts, use_ao_symmetry=False)
kmf.xc = 'pbe'
kmf.kernel()

#
# # The mean-field object with k-point symmetry can be converted back to
# # the corresponding non-symmetric mean-field object
# #

# kmf = kmf.to_khf()
# kmf.kernel(kmf.make_rdm1())

# #
# # Second order SCF solver can be used in the PBC SCF code the same way in the
# # molecular calculation
# #
# kmf = scf.KRHF(cell, kpts).newton()
# kmf.kernel()

# #
# #KUHF
# #
# kumf = scf.KUHF(cell, kpts)
# kumf.kernel()

# #
# # The mean-field object with k-point symmetry can be converted back to
# # the corresponding non-symmetric mean-field object
# #
# kumf = kumf.to_khf()
# kumf.kernel(kumf.make_rdm1())

# #
# #KUHF with smearing
# #
# cell.spin = 2 * 2**3 #assume S=1 in each cell
# kumf = scf.KUHF(cell, kpts)
# #fix_spin: 
# #   if True:
# #       the final solution will have the same spin as input
# #   if False: 
# #       alpha and beta orbitals are sorted together based on energies,
# #       and the final solution can have different spin from input
# #
# #Note: when gap is small, symmetry broken solution is usually the case,
# #      which should be computed by turning off the symmstry options
# kumf = scf.addons.smearing_(kumf, sigma=0.001, method='fermi',fix_spin=True)
# kumf.kernel()


WARNING!
  Very diffused basis functions are found in the basis set. They may lead to severe
  linear dependence and numerical instability.  You can set  cell.exp_to_discard=0.1
  to remove the diffused Gaussians whose exponents are less than 0.1.



System: uname_result(system='Linux', node='frea', release='4.19.87-amd64-nvm', version='#4 SMP Thu Dec 12 07:06:36 EST 2019', machine='x86_64')  Threads 96
Python 3.10.16 (main, Dec 11 2024, 16:24:50) [GCC 11.2.0]
numpy 2.2.1  scipy 1.14.1  h5py 3.12.1
Date: Fri Dec 27 02:03:59 2024
PySCF version 2.7.0
PySCF path  /nessa/users/zhanghao/dev/pyscf
GIT HEAD (branch master) 0f17748077eb042b4e3f7d6be2572d0c755f66e4

[CONFIG] conf_file None
[INPUT] verbose = 5
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 2
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Si     0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 Si     1.346756098700   1.346756098700   1.346756098700 A

<class 'pyscf.pbc.scf.khf_ksymm.KsymAdaptedKRHF'> does not have attributes  xc


Ewald components = 5.45104171915183e-09, -0.167084437254625, 0.0544895952564158
    CPU time for vj and vk    461.95 sec, wall time      4.93 sec
E1 = (2.320837500200521-3.3376188662835208e-18j)  E_coul = (-2.0937165337999044-9.01933160422526e-18j)
init E= -8.23935865092546
    CPU time for initialize scf    494.67 sec, wall time      5.27 sec
HOMO = 0.351400205961  LUMO = 0.425346169548
     k-point                  mo_energy
   0 ( 0.000  0.000  0.000)   [-0.96822209  0.35140021  0.35140021  0.35140021] [0.4662654  0.49904617 0.49904617 0.49904617]
   1 (-0.000 -0.000  0.500)   [-0.54038733 -0.3188218   0.25667836  0.25667836] [0.42534617 0.61946508 0.61946508 0.95126537]
   2 ( 0.000  0.500 -0.000)   [-0.54038733 -0.3188218   0.25667836  0.25667836] [0.42534617 0.61946508 0.61946508 0.95126537]
   3 ( 0.000  0.500  0.500)   [-0.34119879 -0.34119879  0.12922504  0.12922504] [0.46991916 0.46991916 0.86399419 0.86399419]
   4 ( 0.500  0.000  0.000)   [-0.54038733 -0.3188218   0.2566783

np.float64(-7.523159411002124)

In [8]:
fock = kmf.get_fock()

Ewald components = 5.45104171915183e-09, -0.167084437254625, 0.0544895952564158
    CPU time for vj and vk    391.10 sec, wall time      4.10 sec


(3, 8, 8)